In [6]:

# Step 2: Import necessary libraries
import folium
import pandas as pd
from folium.plugins import MarkerCluster  # For clustering markers (optional)
from folium.plugins import MousePosition  # Optional: To show mouse position
from folium.features import DivIcon  # To display custom markers

# Step 3: Fetch and read the CSV data
from urllib.request import urlopen
import io

# URL of the SpaceX launch geo data
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the data from the URL
response = urlopen(URL)

# Convert the binary data into a Pandas DataFrame
spacex_csv_file = io.BytesIO(response.read())
spacex_df = pd.read_csv(spacex_csv_file)

# Step 4: Filter relevant columns and group by Launch Site
# Extracting only relevant columns
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Group by launch site to ensure unique launch sites
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# Select only relevant columns (Launch Site, Lat, Long)
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Display the processed DataFrame (optional step for debugging)
launch_sites_df

# Step 5: Create the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Step 6: Add a Circle and a Marker for NASA Johnson Space Center
# Add a circle to represent NASA Johnson Space Center
folium.Circle(
    location=nasa_coordinate,
    radius=1000,  # Adjust radius as needed
    color='#d35400',
    fill=True,
    fill_color='#d35400'
).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)

# Add a custom marker with a text label for NASA Johnson Space Center
folium.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC'
    )
).add_to(site_map)

# Step 7: Add circles and markers for all launch sites from the dataframe
for _, row in launch_sites_df.iterrows():
    site_lat = row['Lat']
    site_long = row['Long']
    site_name = row['Launch Site']
    
    # Create a circle at each launch site's coordinates
    folium.Circle(
        location=[site_lat, site_long],
        radius=1000,  # You can adjust the radius as needed
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    # Add a custom text marker for each launch site
    folium.Marker(
        location=[site_lat, site_long],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# Step 8: Optionally, add Marker Clustering for better map navigation
marker_cluster = MarkerCluster().add_to(site_map)
for _, row in launch_sites_df.iterrows():
    site_lat = row['Lat']
    site_long = row['Long']
    site_name = row['Launch Site']
    
    # Add launch sites to the MarkerCluster (instead of individually)
    folium.Marker(
        location=[site_lat, site_long],
        popup=folium.Popup(site_name),
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(marker_cluster)

# Step 9: Show the map (the map will display in Jupyter, Colab, or compatible environments)
site_map
